In [ ]:
from __future__ import division, print_function

import GPy
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# parameter grid
def combinations(arrays):
    """Return a single array with combinations of parameters
    
    Parameters
    ----------
    arrays - list of np.array
    
    Returns
    -------
    array - np.array
        An array that contains all combinations of the input arrays
    """
    return np.array(np.meshgrid(*arrays)).T.reshape(-1, len(arrays))


# x_min, x_max, n_samples
grid_param = [(-1, 1, 10),
              (-1, 1, 10)]

grid = combinations([np.linspace(*x) for x in grid_param])

In [ ]:
def line_search_bisection(f, bound, accuracy):
    """Maximize c so that constraint fulfilled.
    
    Parameters
    ----------
    f - callable
        A function that takes a scalar value and return True if
        the constraint is fulfilled, False otherwise.
    bound - list
        Interval within which to search
    accuracy - float
        The interval up to which the algorithm shall search
        
    Returns
    -------
    c - float
        The maximum value c so that the constraint is fulfilled    
    """
    # Break if lower bound does not fulfill constraint
    if not f(bound[0]):
        return None
    
    if f(bound[1]):
        return bound[1]
    
    while bound[1] - bound[0] > accuracy:
        mean = (bound[0] + bound[1]) / 2
        
        if f(mean):
            bound[0] = mean
        else:
            bound[1] = mean
    
    return bound[0]

In [ ]:
def lqr(A, B, Q, R):
    """
    Compute the continuous time LQR-controller. 
    
    Parameters
    ----------
    A - np.array
    B - np.array
    Q - np.array
    R - np.array
     
    Returns
    -------
    K - np.array
        Controller matrix
    P - np.array
        Cost to go matrix
    """
 
    #first, try to solve the ricatti equation
    P = sp.linalg.solve_continuous_are(A, B, Q, R)
     
    #compute the LQR gain
    K = np.linalg.solve(R, B.T.dot(P))
     
    return K, P


def quadratic_lyapunov_function(x, P):
    """
    Compute V(x) for quadratic Lyapunov function
    
    V(x) = x.T P x
    
    Equivalent, but slower implementation:
    np.array([ xi.dot(p.dot(xi.T)) for xi in x])
    
    Parameters
    ----------
    x - np.array
        2d array that has a vector x on each row
    P - np.array
        2d cost matrix for lyapunov function

    Returns
    -------
    V - np.array
        1d array with V(x)
    dV - np.array
        2d array with dV(x)/dx on each row
    """
    return np.sum(x.dot(P) * x, axis=1), x.dot(P)

In [ ]:
A = np.array([[0, 1],
              [0, 0]])

B = np.array([[0],
              [1]])

Q = np.diag([1, 0.01])
R = np.array([[0.1]])

K, P = lqr(A, B, Q, R)

def control_law(x):
    return x.dot(K.T)

def true_dynamics(x, u):
    u = np.asarray(u)
    np.clip(u, -0.5, 0.5, out=u)
    x = np.asarray(x)
    return x.dot(A.T) + u.dot(B.T)


# Initial safe set
S0 = np.linalg.norm(grid, axis=1) < 0.5
    
    
    

In [ ]:
V, dV = quadratic_lyapunov_function(grid, P)

In [ ]:
def find_max_levelset(S, V, accuracy):
    """
    Find maximum level set of V in S.
    
    Parameters
    ----------
    S - boolean array
        Elements are True if V_dot <= L tau
    V - np.array
        1d array with values of Lyapunov function.
    """
    
    def levelset_is_safe(c):
        """
        Return true if V(c) is subset of S
        
        Parameters
        ----------
        c: float
            The level set value
            
        Returns:
        safe: boolean
        """
        return np.all(S[V <= c])
    
    return line_search_bisection(levelset_is_safe,
                                 [0, np.max(V)],
                                 accuracy)
        